# ようこそ

ここでは、DMPに入力いただいた情報に基づき、研究活動の支援、およびデータ管理品質向上のためにデータガバナンス機能が提供するワークフローを作成します。

![maDMP_to_workflow](../files/images/maDMP_to_workflow.jpg)


# maDMPを実行する

## 1. DMPの情報からワークフローを作成する
このmaDMPには、作成いただいたDMPの情報の中で、ワークフローの作成に必要な情報が既に埋め込まれています。  
以下のセルを**上から順に全て実行**してください。セルの実行についてはマニュアルを参考にしてください。  

In [ ]:
# DMP情報
field = '%v'
dataSize = '%v'
datasetStructure = '%v'
useDocker = '%v'

In [ ]:
# ワークフローテンプレート取得
clone_url = 'https://github.com/NII-DG/workflow-template'+'.git'
workflow_path = '/home/jovyan/WORKFLOWS'
!git clone $clone_url $workflow_path
!rm -rf $workflow_path/.git

In [ ]:
import requests
from WORKFLOWS.except_class import query_err
from WORKFLOWS.utils import display_util
from WORKFLOWS.utils.param_json import param_json

# WORKFLOWS/FLOW/param_files/params.json の　"siblings": "ginHttp"　"ginSsh":を更新する。
origin_urls = !git config remote.origin.url
url = origin_urls[0]
try:
    param_json.update_param_url(url)
except requests.exceptions.RequestException:
    display_util.display_err("現在、通信不良が発生しています。")
except query_err.QueryError:
    display_util.display_err("不正なクエリが発行されました。リポジトリIDが不正なものです。")

In [ ]:
# dmp.jsonに"fields"プロパティがある想定
import os
import glob
import shutil

path_flows = os.path.join('WORKFLOWS', 'FLOW')

templates = glob.glob(os.path.join(path_flows, '**'), recursive=True)

# 選択外の分野のセクション群を削除
for tmpl in templates:
    file = os.path.basename(tmpl)
    if not os.path.isdir(tmpl) and os.path.splitext(file)[1] == '.ipynb':
        if 'base_' not in file and field not in file:
            os.remove(tmpl)

In [ ]:
# モニタリング準備
import os
import json

path_params = os.path.join('WORKFLOWS', 'FLOW', 'param_files', 'monitoring_params.json')
monitoring_dict = {
    'dataSize': dataSize,
    'datasetStructure': datasetStructure
    # モニタリング項目を追加する場合は以下に追記してください
    # e.g. 'dirStructure': 'withCode'
}

with open(path_params, 'w') as f:
    json.dump(monitoring_dict, f, indent=4)

## 2. ワークフロー図を準備する

次に、ワークフローが俯瞰できる図を表示する準備をします。  
以下のセルを実行してください。  
実行完了後、表示されるリンクをクリックして研究ワークフロー機能トップページへと遷移してください。

In [ ]:
import os
import nbformat
import urllib.parse
from IPython.display import display, HTML, Javascript

flow_path = 'WORKFLOWS/base_FLOW.ipynb'
rcos_binder_url = 'https://jupyter.cs.rcos.nii.ac.jp/'
flow_path_notebook = 'notebooks/WORKFLOWS/base_FLOW.ipynb'
# flow_path_lab = 'lab/tree/WORKFLOW/FLOW/base_FLOW.ipynb'

nb = nbformat.read(flow_path, 4)

title1 = """## ワークフロー図を作成する

以下のセルを実行してワークフロー図を作成できます。  
すでに実行したタスクを確認したい場合は再度実行してください。"""

code1 = """import os
from IPython.display import display, SVG, Javascript

nb_utils_path   = os.path.join(os.environ['HOME'], 'WORKFLOWS/FLOW/util/scripts/nb_utils.py')

basic_path      = os.path.join(os.environ['HOME'], 'WORKFLOWS/images')
basic_path_diag = os.path.join(basic_path, 'notebooks.diag')
basic_path_svg  = os.path.join(basic_path, 'notebooks.svg')

os.chdir(os.environ['HOME'])
!python3 $nb_utils_path $basic_path_diag

display(SVG(filename=basic_path_svg))
display(Javascript('IPython.notebook.save_checkpoint();'))"""

title2 = """ワークフロー図の左側にあるノードはワークフローのフェーズとその流れを表しており、右側にあるノードは個々の支援するタスクを表しています。  
各タスクを実行するには、実行したいタスクのノードのリンクをクリックしてください。"""

title3 = """## GIN-forkへ戻る

以下のセルを実行し、表示されるURLをクリックするとGIN-forkへアクセスできます。  
リンクをクリックして404エラーが表示された場合は、リポジトリ名が更新されている可能性があるため以下のセルを再度実行して下さい。"""

code2 = """import os
import subprocess
from IPython.display import display, Javascript, clear_output

# リモートリポジトリのURLを最新化する
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.update_repo_url()
os.chdir(os.environ['HOME'])

# GIN-forkへのアクセスリンクを表示する
clear_output()
print('以下のリンクをクリックしてGIN-forkへ遷移してください。新規タブで開きます。\\n')
print(subprocess.getoutput('git config --get remote.origin.url'))
display(Javascript('IPython.notebook.save_checkpoint();'))"""

# ワークフロー図作成セルが無い場合のみbase_FLOW.ipynbにセルを追加する
if '## ワークフロー図を作成する' in str(nb):
    print('ワークフロー図はすでに準備されています。以下のリンクをクリックしてください。')
else:
    nb["cells"] += [
        nbformat.v4.new_markdown_cell(title1),
        nbformat.v4.new_code_cell(code1),
        nbformat.v4.new_markdown_cell(title2),
        nbformat.v4.new_markdown_cell(title3),
        nbformat.v4.new_code_cell(code2)
    ]
    with open(flow_path, "w") as f:
        nbformat.write(nb, f)
    print('ワークフロー図の準備が完了しました。以下のリンクをクリックしてください。')
display(HTML("<a href='./WORKFLOWS/base_FLOW.ipynb'>研究ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))